In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Norman Wells,CA,2022-03-24 06:19:16,65.2820,-126.8329,-7.56,77,40,0.00
1,1,Arraial Do Cabo,BR,2022-03-24 06:19:44,-22.9661,-42.0278,74.77,81,6,17.56
2,2,Amapa,BR,2022-03-24 06:19:45,1.0000,-52.0000,72.18,100,100,2.01
3,3,Albany,US,2022-03-24 06:15:10,42.6001,-73.9662,37.02,87,100,5.99
4,4,Waitati,NZ,2022-03-24 06:19:46,-45.7500,170.5667,58.80,70,97,7.58


In [2]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [25]:
city_data_df.count()

City_ID       23
City          23
Country       23
Date          23
Lat           23
Lng           23
Max Temp      23
Humidity      23
Cloudiness    23
Wind Speed    23
dtype: int64

In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]
# 2. Assign the weights variable to some values.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent wind
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Le Port,RE,2022-03-24 06:19:47,-20.9373,55.2919,85.96,64,93,4.88
9,9,Georgetown,MY,2022-03-24 06:20:56,5.4112,100.3354,89.55,66,20,8.05
15,15,Avarua,CK,2022-03-24 06:24:19,-21.2078,-159.7750,80.65,83,75,10.36
21,21,Carnarvon,AU,2022-03-24 06:24:23,-24.8667,113.6333,89.67,51,100,9.22
22,22,Ewa Beach,US,2022-03-24 06:24:24,21.3156,-158.0072,76.98,67,20,18.01


In [14]:
preferred_cities_df.count()

City_ID       5
City          5
Country       5
Date          5
Lat           5
Lng           5
Max Temp      5
Humidity      5
Cloudiness    5
Wind Speed    5
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max_temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng",]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Le Port,RE,85.96,-20.9373,55.2919,
9,Georgetown,MY,89.55,5.4112,100.3354,
15,Avarua,CK,80.65,-21.2078,-159.7750,
21,Carnarvon,AU,89.67,-24.8667,113.6333,
22,Ewa Beach,US,76.98,21.3156,-158.0072,


In [18]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [24]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location ket for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make requests and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
            
hotel_df


,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Le Port,RE,85.96,-20.9373,55.2919,KORBEY D'OR OUEST
9,Georgetown,MY,89.55,5.4112,100.3354,Cititel Penang
15,Avarua,CK,80.65,-21.2078,-159.7750,Paradise Inn
21,Carnarvon,AU,89.67,-24.8667,113.6333,Hospitality Carnarvon
22,Ewa Beach,US,76.98,21.3156,-158.0072,Base Lodging


In [26]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
                                dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [32]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#store the DataFrame.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]